Complete the code in the blocks below. Do not alter blocks that indicate that they should not be changed.

Look for blocks that say "FILL IN THIS SECTION"

FILL IN THIS SECTION

Author(s): Emilio Cardenas, Deautaun Ross


In this homework we will write a planning system based on STRIPS.

In [ ]:
# Import statements here if needed

In [1]:
# DO NOT CHANGE
# Define a class to represent one action

class Action:
  def __init__(self, name: str, pre: dict, eff: dict) -> None:
    self.name = name
    self.precondition = pre
    self.effect = eff

  def __hash__(self):
    return hash(self.name)

  def __eq__(self, other):
    return self.name == other.name

In [2]:
# DO NOT CHANGE
# Represent Block World (seen in Figure 10.4 of AIMA, pg. 377 in 3rd edition)

# Init
initial_state = {
  'A_on_Table' : True,
  'B_on_Table' : True,
  'C_on_Table' : False,
  'A_on_B' : False,
  'A_on_C' : False,
  'B_on_A' : False,
  'B_on_C' : False,
  'C_on_A' : True,
  'C_on_B' : False,
  'A_Clear' : False,
  'B_Clear' : True,
  'C_Clear' : True
  # Implicitiy Block(A), Block(B), and Block(C) are true but that's not important
  # for this program.
}

goal_state = {
    'C_on_Table': True,
    'B_on_C' : True,
    'A_on_B' : True
}

In [4]:
# FILL IN THIS SECTION

# Here are two examples of Actions:

Move_A_from_B_to_Table = Action(
    name="Move A from B to Table",
    pre={
        'A_on_B' : True,
        'A_Clear' : True
        # Implicitly A is a Block
    },
    eff={
        'A_on_Table' : True,
        'B_Clear' : True,
        'A_on_B' : False
    }
)

Move_A_from_C_to_B = Action(
    name="Move A from C to B",
    pre={
        'A_on_C' : True,
        'A_Clear' : True,
        'B_Clear' : True
        # Implicitly A and B are Blocks
    },
    eff={
        'A_on_B' : True,
        'C_Clear' : True,
        'A_on_C' : False,
        'B_clear' : False
    }
)

all_actions = set([
    Move_A_from_B_to_Table,
    Move_A_from_C_to_B
])

# Use Figure 10.3 in AIMA (pg. 377 of the 3rd Edition) to fill in the
# remainder of the all_actions set. Enforce inequality conditions such as "b != x" etc.

# You can use loops, function calls, or any other approach that you can think
# of, but do not try to hand code all of the remaining actions.

def generate_actions():
    actions = set()

    # Create all possible MoveToTable actions
    for block in ['A', 'B', 'C']:
        for source in ['A', 'B', 'C']:
            if block != source:  # enforcing (b != x)
                actions.add(Action(
                    name=f"Move {block} from {source} to Table",
                    pre={
                        f'{block}_on_{source}': True,
                        f'{block}_Clear': True
                    },
                    eff={
                        f'{block}_on_Table': True,
                        f'{source}_Clear': True,
                        f'{block}_on_{source}': False
                    }
                ))

    # Create all possible Move actions for moving one block onto another
    for block in ['A', 'B', 'C']:
        for source in ['A', 'B', 'C', 'Table']:
            for destination in ['A', 'B', 'C']:
                if block != source and block != destination and source != destination:
                    actions.add(Action(
                        name=f"Move {block} from {source} to {destination}",
                        pre={
                            f'{block}_on_{source}': True,
                            f'{block}_Clear': True,
                            f'{destination}_Clear': True
                        },
                        eff={
                            f'{block}_on_{destination}': True,
                            f'{source}_Clear': (source != 'Table'),  # The table is always clear, so we don't change its state
                            f'{block}_on_{source}': False,
                            f'{destination}_Clear': False
                        }
                    ))
    return actions

# Add generated actions to all_actions set
all_actions.update(generate_actions())

# Print all actions and their thingies
for i, action in enumerate(all_actions):
    print(f"Action {i}: {action.name}")
    print("Preconditions: ")
    for key, value in action.precondition.items():
        print(f"    {key}: {value}")
    print("Effects:")
    for key, value in action.effect.items():
        print(f"    {key}: {value}")



Action 0: Move A from Table to B
Preconditions: 
    A_on_Table: True
    A_Clear: True
    B_Clear: True
Effects:
    A_on_B: True
    Table_Clear: False
    A_on_Table: False
    B_Clear: False
Action 1: Move C from A to Table
Preconditions: 
    C_on_A: True
    C_Clear: True
Effects:
    C_on_Table: True
    A_Clear: True
    C_on_A: False
Action 2: Move B from C to Table
Preconditions: 
    B_on_C: True
    B_Clear: True
Effects:
    B_on_Table: True
    C_Clear: True
    B_on_C: False
Action 3: Move A from C to Table
Preconditions: 
    A_on_C: True
    A_Clear: True
Effects:
    A_on_Table: True
    C_Clear: True
    A_on_C: False
Action 4: Move B from Table to C
Preconditions: 
    B_on_Table: True
    B_Clear: True
    C_Clear: True
Effects:
    B_on_C: True
    Table_Clear: False
    B_on_Table: False
    C_Clear: False
Action 5: Move C from B to Table
Preconditions: 
    C_on_B: True
    C_Clear: True
Effects:
    C_on_Table: True
    B_Clear: True
    C_on_B: False
Action 6

In [5]:
# FILL IN THIS SECTION

def is_legal(action: Action, state: dict):
  """
  Parameters:
    action: one Action
    state: one global state
  Returns:
    True if the Action's preconditions are met in the current state
    False otherwise
  """
  for pre_condition, pre_value in action.precondition.items():
      # If the precondition is not met in the state, return False
      if state.get(pre_condition) != pre_value:
          return False
  return True

In [8]:
# FILL IN THIS SECTION

def write_plan(actions: set, init_state: dict, goal: dict) -> list[Action]:
    """
    Parameters:
        actions: a set of possible actions
        initial_state: the starting world state dictionary
        goal: a dictionary of goal conditions
    Returns:
        A list of actions chosen from actions that ends with the goal conditions being true.
    """
    def apply_action(action: Action, state: dict):
        # Applies an action to the state.
        new_state = state.copy()
        for eff, eff_value in action.effect.items():
            new_state[eff] = eff_value
        return new_state

    def goal_achieved(state: dict, goal_conditions: dict):
        # Checks if the goal state has been achieved.
        return all(state.get(goal) == val for goal, val in goal_conditions.items())

    def backward_chain(plan: list, state: dict, goals: dict):
        # Backward chaining recursive function.
        if goal_achieved(state, goals):
            return True

        for action in actions:
            if any(goal in action.effect for goal in goals) and is_legal(action, state):
                new_state = apply_action(action, state)
                new_goals = {g: goals[g] for g in goals if new_state.get(g) != goals[g]}

                print(f"Trying action: {action.name}")
                if backward_chain(plan, new_state, new_goals):
                    plan.append(action)
                    return True
                else:
                    print(f"Backtracking from action: {action.name}")

        return False

    plan = []
    success = backward_chain(plan, init_state, goal)
    if not success:
        print("No plan could be found to achieve the goal state.")
        return []
    else:
        print("Plan found!\n")
        plan.reverse()  # Reverse the plan since we worked backwards
        return plan

In [9]:
# DO NOT CHANGE

plan = write_plan(all_actions, initial_state, goal_state)
print(f"To stack our blocks we must: {', '.join([p.name for p in plan])}")

Trying action: Move C from A to Table
Trying action: Move A from Table to B
Backtracking from action: Move A from Table to B
Trying action: Move B from Table to C
Trying action: Move A from Table to B
Plan found!

To stack our blocks we must: Move C from A to Table, Move B from Table to C, Move A from Table to B
